In [0]:
#Loading the required libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import MinMaxScaler,Normalizer

In [0]:
class Preprocess(object):
  def __init__(self):
    self.url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00225/Indian%20Liver%20Patient%20Dataset%20(ILPD).csv"
    self.cols = ['age','gender','TB','DB','alkphos','sgpt','sgot','TP','albumin','AG_ratio','Is_liverPatient']
    self.dataset = None
  
  def load_data(self):
    self.dataset = pd.read_csv(self.url,names=self.cols)
    print('shape of df :',self.dataset.shape)
    return self.dataset
  
  def preprocess(self,data):
    #preprocessing the data
    # 1. Removing the duplicate rows
    self.dataset = data.drop_duplicates()
    print('Shape after removing the duplicates :',self.dataset.shape)
    # checking for the Null values, #display no of null values by column
    print(self.dataset.isnull().sum())
    #dropping the 4 rows here
    self.dataset = self.dataset[pd.notnull(self.dataset['AG_ratio'])]
    return None
  
  def encode_label(self):
    #changing the gender attribute to categorical type
    # 0 is for 'Female' and 1 for 'Male
    self.dataset.gender = pd.factorize(self.dataset.gender)[0] + 0.0
    #Encoding the label for proper implementation in network (1 neuron in the output layer)
    # Label 1 means "a liver patient" so encoding as 1
    # Label 2 means "not a liver patient" so encoding as 0
    self.dataset.loc[(self.dataset.Is_liverPatient == 2), 'Is_liverPatient'] = 0
    return None
  
  def scaleData(self):
    #Scaling the attribute values to be between [0,1]
    scaler = MinMaxScaler()
    cols_to_scale = ['age', 'TB', 'DB', 'alkphos', 'sgpt', 'sgot', 'TP', 'albumin', 'AG_ratio']
    Dataset = pd.DataFrame(data=self.dataset)
    self.dataset[cols_to_scale] = scaler.fit_transform(Dataset[cols_to_scale])
    return None
  
  def get_dataset(self):
    return self.dataset

In [0]:
prep = Preprocess()
dataset = prep.load_data()
prep.preprocess(dataset)
prep.encode_label()
prep.scaleData()
dataset = prep.get_dataset()

shape of df : (583, 11)
Shape after removing the duplicates : (570, 11)
age                0
gender             0
TB                 0
DB                 0
alkphos            0
sgpt               0
sgot               0
TP                 0
albumin            0
AG_ratio           4
Is_liverPatient    0
dtype: int64


In [0]:
#Splitting the dataset into train and test splits
# Taking the label out of the original preprocessed dataset
X = dataset[['age', 'gender', 'TB', 'DB', 'alkphos', 'sgpt', 'sgot', 'TP', 'albumin', 'AG_ratio']]
y = dataset['Is_liverPatient']

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.70)  #test_size = 1-0.7 = 0.3
#Changing them into numpy array as the last step
X_train = X_train.to_numpy().astype(np.float32)
y_train = y_train.to_numpy().astype(np.int32)
X_test = X_test.to_numpy().astype(np.float32)
y_test = y_test.to_numpy().astype(np.int32)
print('X_train shape :',X_train.shape)
print('y_train shape :',y_train.shape)
print('X_test shape :',X_test.shape)
print('y_test shape :',y_test.shape)

X_train shape : (396, 10)
y_train shape : (396,)
X_test shape : (170, 10)
y_test shape : (170,)


In [0]:
#main procedure call

#creating the tensors out of the numpy arrays
X_train = tf.Variable(X_train, dtype=tf.float32, trainable=False)
y_train = tf.Variable(y_train, dtype=tf.int32, trainable=False)
X_test = tf.Variable(X_test, dtype=tf.float32, trainable=False)
y_test = tf.Variable(y_test, dtype=tf.int32, trainable=False)

#intializing the network parameters
input_dimension = X_train.shape[1]
num_classes = 1
#layer_activations = ["Relu", "Relu", "Softmax"]
layer_activations = ["Relu", "Softmax"]
nodes_in_layers = [5, num_classes]

#intializing the hyperparameters
batch_size = 50
epochs = 10
alpha = 0.1

#creating the tensor datasets for incorporating the batches
train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(100).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(batch_size)


In [0]:
weights = []
biases = []
trainable_vars = []

In [0]:
for i,nodes in enumerate(nodes_in_layers):
  if i == 0:
    w = tf.Variable(np.random.randn(*(input_dimension,nodes)), dtype=tf.float32,trainable=True)
    b = tf.Variable(np.zeros(shape=(nodes,)),dtype=tf.float32, trainable=True)
  else:
    w = tf.Variable(np.random.randn(*(nodes_in_layers[i-1],nodes)),dtype=tf.float32 ,trainable=True)
    b = tf.Variable(np.zeros(shape=(nodes,)),dtype=tf.float32, trainable=True)
  weights.append(w)
  biases.append(b)

In [0]:
for i in range(len(weights)):
  print(weights[i].shape)

(10, 5)
(5, 1)


In [0]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.SGD(learning_rate=alpha)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

In [0]:
def predict(X):
  for i in range(len(weights)):
    if layer_activations[i] == 'Relu':
      X = tf.nn.relu(tf.add(tf.matmul(X,weights[i]),biases[i]))
    elif layer_activations[i] == 'Sigmoid':
      X = tf.nn.sigmoid(tf.add(tf.matmul(X,weights[i]),biases[i]))
    elif layer_activations[i] == 'Softmax':
      X = tf.nn.softmax(tf.add(tf.matmul(X,weights[i]),biases[i]))
    else:
     assert 2 == 4
  return X

In [0]:
def get_trainableVars():
  for i in range(len(weights)):
    trainable_vars.append(weights[i])
  for i in range(len(biases)):
    trainable_vars.append(biases[i])
  return trainable_vars

Implementing the training in multiple different ways:

1. Gradient Descent (the simplest one)
2. SGD (Stocastic Gradient Descent)
   (using 2 approaches i.e a) apply_gradients b) minimise()
3. Adam
  (using 2 approaches i.e a) apply_gradients b) minimise()

In [0]:
# Gradient Descent (the simplest one)
@tf.function
def train(X,y):
  with tf.GradientTape(persistent=True) as tape:
    predictions = predict(X)
    loss = loss_object(y,predictions)
    for i in range(len(weights)):
      dloss_dw,dloss_db = tape.gradient(loss,[weights[i],biases[i]])
      assert dloss_dw != None or dloss_db != None
      weights[i].assign_sub(alpha * dloss_dw)
      biases[i].assign_sub(alpha * dloss_db)
    train_loss(loss)
    train_accuracy(y,predictions)

In [0]:
# SGD (Stocastic Gradient Descent) ==> a) apply_gradients
@tf.function
def train(X,y):
  with tf.GradientTape() as tape:
    predictions = predict(X)
    loss = loss_object(y,predictions)
    trainable_vars = get_trainableVars()
  gradients = tape.gradient(loss,trainable_vars)
  optimizer.apply_gradients(zip(gradients,trainable_vars))
  train_loss(loss)
  train_accuracy(y,predictions)
  

In [0]:
def loss():
  return loss_object(y_train,predict(X_train))

In [0]:
# SGD (Stocastic Gradient Descent) ==> b) minimise()

def train(X,y):
  with tf.GradientTape() as tape:
    predictions = predict(X)
    #loss = loss_object(y,predictions)
    trainable_vars = get_trainableVars()
  optimizer.minimize(loss,trainable_vars)    #here the loss parameter needs to be callable, therefore the loss() function is created. 
  train_loss(loss_object(y,predictions))
  train_accuracy(y,predictions)

In [0]:
@tf.function
def test(X,y):
  predictions = predict(X)
  t_loss = loss_object(y,predictions)
  test_loss(t_loss)
  test_accuracy(y,predictions)

In [0]:
print(tf.math.confusion_matrix(labels=y_train,predictions=predict(X_train)))

tf.Tensor(
[[  0 113]
 [  0 283]], shape=(2, 2), dtype=int32)


In [0]:
print(tf.math.confusion_matrix(labels=y_test,predictions=predict(X_test)))

tf.Tensor(
[[  0  49]
 [  0 121]], shape=(2, 2), dtype=int32)


In [0]:
# starting the training
for epoch in range(epochs):
  #reset the metric in the beginning of each epoch 
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for x,label in train_ds:
    train(x,label)
  
  for x,label in test_ds:
    test(x,label)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

Epoch 1, Loss: 0.5981529951095581, Accuracy: 71.51087188720703, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 2, Loss: 0.5985878109931946, Accuracy: 71.46739959716797, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 3, Loss: 0.5981529951095581, Accuracy: 71.51087188720703, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 4, Loss: 0.5983704328536987, Accuracy: 71.4891357421875, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 5, Loss: 0.5972834229469299, Accuracy: 71.59782409667969, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 6, Loss: 0.5981530547142029, Accuracy: 71.51087188720703, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 7, Loss: 0.5977182388305664, Accuracy: 71.55435180664062, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 8, Loss: 0.5972834229469299, Accuracy: 71.59782409667969, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 9, Loss: 0.5990225672721863, Accuracy: 71.42391204833984, Test Loss: 0.6032

In [0]:
# starting the training
for epoch in range(epochs):
  #reset the metric in the beginning of each epoch 
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for x,label in train_ds:
    train(x,label)
  
  for x,label in test_ds:
    test(x,label)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

Epoch 1, Loss: 0.5979356169700623, Accuracy: 71.5326156616211, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 2, Loss: 0.5979356169700623, Accuracy: 71.53260040283203, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 3, Loss: 0.5983704328536987, Accuracy: 71.48912811279297, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 4, Loss: 0.5977182388305664, Accuracy: 71.55435180664062, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 5, Loss: 0.5985878109931946, Accuracy: 71.46739196777344, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 6, Loss: 0.5981529951095581, Accuracy: 71.51087188720703, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 7, Loss: 0.5983704328536987, Accuracy: 71.48912811279297, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 8, Loss: 0.5981529951095581, Accuracy: 71.51087188720703, Test Loss: 0.6032617688179016, Test Accuracy: 71.0
Epoch 9, Loss: 0.5983704328536987, Accuracy: 71.48912811279297, Test Loss: 0.6032